In [16]:
import pandas as pd
import json 
import requests

In [17]:
API_KEY = "cqt_rQBpT889pBtdJRyj4m6vqKDC9x9R"
CHAIN_ID = "1"  

In [18]:
with open("wallet_address.txt", "r") as file:
    wallet_addresses = [line.strip() for line in file.readlines()]

# Get data token holdings
def get_wallet_holdings(wallet_address):
    url = f"https://api.covalenthq.com/v1/{CHAIN_ID}/address/{wallet_address}/balances_v2/"
    params = {"key": API_KEY}
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error while query {wallet_address}: {response.status_code}")
        return None



In [25]:
wallet_data=[]

for wallet in wallet_addresses:
    data=get_wallet_holdings(wallet)
    # print(json.dumps(data,indent=4))
    wallet_data.append(data)

with open("wallet_data.json","w") as f:
    json.dump(wallet_data,f,indent=4)


In [27]:
with open("wallet_data.json","r") as f:
    wallet_data=json.load(f)

In [ ]:
# Save data
wallet_data = []

for wallet in wallet_addresses:
    data = get_wallet_holdings(wallet)
    if data and "data" in data:
        total_usd_value = 0
        tokens = []

        for item in data["data"]["items"]:
            token_name = item["contract_name"]
            token_symbol = item["contract_ticker_symbol"]
            token_balance = int(item["balance"]) / (10 ** item["contract_decimals"])
            token_price = item["quote"]
            token_value = token_balance * token_price
            
            total_usd_value += token_value
            tokens.append({
                "Token": token_name,
                "Symbol": token_symbol,
                "Balance": token_balance,
                "Price (USD)": token_price,
                "Value (USD)": token_value
            })

        # Tính phân bổ tài sản (%)
        for token in tokens:
            token["Percentage (%)"] = (token["Value (USD)"] / total_usd_value) * 100 if total_usd_value > 0 else 0

        wallet_data.append({
            "Wallet": wallet,
            "Total Value (USD)": total_usd_value,
            "Tokens": tokens
        })

# 🔹 Xuất dữ liệu ra file CSV
df_list = []
for wallet in wallet_data:
    for token in wallet["Tokens"]:
        df_list.append({
            "Wallet": wallet["Wallet"],
            "Token": token["Token"],
            "Symbol": token["Symbol"],
            "Balance": token["Balance"],
            "Price (USD)": token["Price (USD)"],
            "Value (USD)": token["Value (USD)"],
            "Percentage (%)": token["Percentage (%)"]
        })

df = pd.DataFrame(df_list)
df.to_csv("wallet_analysis.csv", index=False)

print("✅ Phân tích hoàn tất! Kết quả đã lưu vào `wallet_analysis.csv` 🎉")
